## Accelerating Python code with Numba

In [1]:
import math
import random
import numpy as np
from numba import jit, vectorize, float64
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

### Random walk

In [2]:
def step():
    return 1. if random.random() > .5 else -1.

In [3]:
def walk(x):
    n = len(x)
    dx = 1. / n
    for i in range(n - 1):
        x_new = x[i] + dx * step()
        if x_new > 5e-3:
            x[i + 1] = 0.
        else:
            x[i + 1] = x_new

In [4]:
n = 100000
x = np.zeros(n)

In [5]:
walk(x)

In [6]:
plt.plot(x)

In [7]:
%%timeit x = np.zeros(n)
walk(x)

10 loops, best of 3: 47.9 ms per loop

In [8]:
@jit(nopython=True)
def step_numba():
    return 1. if random.random() > .5 else -1.

In [9]:
@jit(nopython=True)
def walk_numba(x):
    n = len(x)
    dx = 1. / n
    for i in range(n - 1):
        x_new = x[i] + dx * step_numba()
        if x_new > 5e-3:
            x[i + 1] = 0.
        else:
            x[i + 1] = x_new

In [10]:
%%timeit x = np.zeros(n)
walk_numba(x)

The slowest run took 33.25 times longer than the fastest. This could mean that an intermediate result is being cached
1000 loops, best of 3: 1.59 ms per loop

### Universal functions

In [11]:
x = np.random.rand(10000000)
%timeit np.cos(2*x**2 + 3*x + 4*np.exp(x**3))

1 loops, best of 3: 689 ms per loop

In [12]:
@vectorize
def kernel(x):
    return np.cos(2*x**2 + 3*x + 4*np.exp(x**3))

In [13]:
kernel(1.)

-0.98639139715432589

In [14]:
%timeit kernel(x)

1 loops, best of 3: 324 ms per loop

In [15]:
import numexpr
%timeit numexpr.evaluate('cos(2*x**2 + 3*x + 4*exp(x**3))')

10 loops, best of 3: 122 ms per loop

In [16]:
numexpr.detect_number_of_cores()

4